In [ ]:
!pip install -q gdown


In [ ]:
file_id = "1QdLkRuVFErTcmO9ZKNLnzEG1IMZBiDJF"
!gdown --id {file_id} --output dataset.zip


In [ ]:
import zipfile
import os

zip_path = "dataset.zip"
extract_path = "Signal_Dataset"  

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Unzipped successfully to:", extract_path)


In [3]:
import os
import shutil
import random
import xml.etree.ElementTree as ET
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Set your dataset paths
IMG_DIR = "Signal_Dataset/images"
ANN_DIR = "Signal_Dataset/annotations"
YOLO_DATASET_DIR = "YOLO_Format_Dataset"

# Define class labels
CLASSES = ["trafficlight", "stop", "speedlimit", "crosswalk"]
CLASS_TO_ID = {cls: i for i, cls in enumerate(CLASSES)}


In [4]:
def convert_bbox(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x_center = (box[0] + box[1]) / 2.0
    y_center = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    return x_center * dw, y_center * dh, w * dw, h * dh

def convert_annotation(img_path, xml_path):
    image = Image.open(img_path)
    w, h = image.size
    yolo_boxes = []

    tree = ET.parse(xml_path)
    root = tree.getroot()

    for obj in root.iter('object'):
        cls = obj.find('name').text.lower()
        if cls not in CLASS_TO_ID:
            continue
        cls_id = CLASS_TO_ID[cls]
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text),
             float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bbox = convert_bbox((w, h), b)
        yolo_boxes.append(f"{cls_id} {' '.join(map(lambda x: format(x, '.6f'), bbox))}")
    return yolo_boxes


In [5]:
def prepare_yolo_dataset(split_ratio=0.2):
    all_images = sorted(os.listdir(IMG_DIR))
    train_files, val_files = train_test_split(all_images, test_size=split_ratio, random_state=42)

    for split, files in zip(['train', 'val'], [train_files, val_files]):
        for folder in ['images', 'labels']:
            os.makedirs(f"{YOLO_DATASET_DIR}/{split}/{folder}", exist_ok=True)

        for file in tqdm(files, desc=f"Preparing {split} set"):
            img_path = os.path.join(IMG_DIR, file)
            ann_path = os.path.join(ANN_DIR, file.replace(".png", ".xml"))

            yolo_labels = convert_annotation(img_path, ann_path)
            with open(f"{YOLO_DATASET_DIR}/{split}/labels/{file.replace('.png', '.txt')}", 'w') as f:
                f.write("\n".join(yolo_labels))

            shutil.copy(img_path, f"{YOLO_DATASET_DIR}/{split}/images/{file}")

prepare_yolo_dataset()


Preparing val set: 100%|██████████| 176/176 [00:03<00:00, 52.18it/s]


In [7]:
yaml = f"""
path: {YOLO_DATASET_DIR}
train: train/images
val: val/images

names:
"""
for i, cls in enumerate(CLASSES):
    yaml += f"  {i}: {cls}\n"

with open(f"{YOLO_DATASET_DIR}/data.yaml", "w") as file:
    file.write(yaml.strip())


In [ ]:
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.1 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")
model.train(data=f"{YOLO_DATASET_DIR}data.yaml", epochs=50, imgsz=640, batch=16,patience=10)

# Evaluate
metrics = model.val()


100%|██████████| 21.5M/21.5M [00:00<00:00, 364MB/s]

Ultralytics 8.3.145 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, pretrained=True, pro

  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  8                  -1  1   1838080  ultralytics.nn.modules.block.C2f             [512, 512, 1, True]           
  9                  -1  1    656896  ultralytics.nn.modules.block.SPPF            [512, 512, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  1    591360  ultralytics.nn.modules.block.C2f             [768, 256, 1]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  1    148224  ultralytics.nn.modules.block.C2f             [384,

train: Scanning /content/YOLO_Format_Dataset/train/labels.cache... 701 images, 0 backgrounds, 0 corrupt: 100%|██████████| 701/701 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

                   all        176        260      0.948      0.922      0.943      0.803

50 epochs completed in 0.276 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 22.5MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.145 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,127,132 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:03<00:00,  1.67it/s]


                   all        176        260      0.934      0.928      0.944      0.804
          trafficlight         23         44      0.922      0.818      0.856      0.584
                  stop         28         28      0.896      0.964      0.982      0.904
            speedlimit        124        141      0.984          1      0.988      0.926
             crosswalk         41         47      0.936      0.931      0.948      0.802
Speed: 0.3ms preprocess, 4.1ms inference, 0.0ms loss, 4.9ms postprocess per image
Results saved to runs/detect/train3
Ultralytics 8.3.145 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,127,132 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2710.7±638.9 MB/s, size: 231.5 KB)


val: Scanning /content/YOLO_Format_Dataset/val/labels.cache... 176 images, 0 backgrounds, 0 corrupt: 100%|██████████| 176/176 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:04<00:00,  2.49it/s]


                   all        176        260      0.943      0.917      0.943      0.805
          trafficlight         23         44       0.93      0.795      0.857      0.579
                  stop         28         28      0.923      0.964       0.98      0.905
            speedlimit        124        141      0.986      0.998      0.988      0.927
             crosswalk         41         47      0.935      0.912      0.948      0.808
Speed: 2.1ms preprocess, 7.8ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/train32


In [ ]:
print("\n📊 Evaluation Metrics:")
print(f"mAP@0.5: {metrics.box.map50:.3f}")
print(f"mAP@0.5:0.95: {metrics.box.map:.3f}")
print(f"Precision: {metrics.box.precision:.3f}")
print(f"Recall: {metrics.box.recall:.3f}")

In [ ]:
from ultralytics import YOLO
import cv2
import os
from pathlib import Path

# ------------------------
# ✅ Configuration Section
# ------------------------

# Path to your trained model
MODEL_PATH = "yolov8small.pt"  # or wherever your model is saved

# Path to the folder containing images
IMAGE_FOLDER = r"YOLO_Format_Dataset\val\images"  # replace with your folder name


SAVE_FOLDER = "Small_model_results"
os.makedirs(SAVE_FOLDER, exist_ok=True)

# -------------------------
# ✅ Load Model and Predict
# -------------------------

# Load the trained model
model = YOLO(MODEL_PATH)

# Get all image paths from folder
image_paths = list(Path(IMAGE_FOLDER).glob("*.png")) + list(Path(IMAGE_FOLDER).glob("*.png"))

# Run prediction and show/save results
for img_path in image_paths:
    # Run inference
    results = model(img_path)

    # Get annotated frame with bounding boxes
    annotated_image = results[0].plot()

    # Show result in a window (500ms)
    cv2.imshow("Prediction", annotated_image)
    cv2.waitKey(500)  # show for 500 milliseconds

    # Save result to disk
    output_path = Path(SAVE_FOLDER) / img_path.name
    cv2.imwrite(str(output_path), annotated_image)

cv2.destroyAllWindows()
print(f"\n✅ All predictions saved to: {SAVE_FOLDER}")



image 1/1 c:\Users\afraz\Desktop\Nexa_eval\Task_05\YOLO_Format_Dataset\val\images\road102.png: 448x640 1 speedlimit, 168.4ms
Speed: 2.4ms preprocess, 168.4ms inference, 0.9ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 c:\Users\afraz\Desktop\Nexa_eval\Task_05\YOLO_Format_Dataset\val\images\road107.png: 640x512 1 speedlimit, 204.1ms
Speed: 2.0ms preprocess, 204.1ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 c:\Users\afraz\Desktop\Nexa_eval\Task_05\YOLO_Format_Dataset\val\images\road119.png: 448x640 1 speedlimit, 167.1ms
Speed: 2.2ms preprocess, 167.1ms inference, 0.6ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 c:\Users\afraz\Desktop\Nexa_eval\Task_05\YOLO_Format_Dataset\val\images\road120.png: 480x640 1 speedlimit, 193.8ms
Speed: 1.8ms preprocess, 193.8ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 c:\Users\afraz\Desktop\Nexa_eval\Task_05\YOLO_Format_Dataset\val\images\road124.png: 640x544 

KeyboardInterrupt: 

: 